In [8]:
from sklearn.cluster import KMeans
import numpy as np

def kmeans_clustering(uav_positions, num_clusters):
    """Perform K-Means clustering on UAV positions."""
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    labels = kmeans.fit_predict(uav_positions)
    return labels, kmeans.cluster_centers_
def gwo_select_cluster_heads(cluster_members, energy_levels, mobility_factors):
    """Optimize CH selection using Grey Wolf Optimization."""
    num_wolves = 5  # Number of search agents
    alpha, beta, delta = None, None, None  # Best three solutions

    for _ in range(50):  # Iterations
        for wolf in range(num_wolves):
            # Fitness function = High Energy & Low Mobility
            fitness = energy_levels / (1 + mobility_factors)
            best_wolf = np.argmax(fitness)

            # Update alpha, beta, delta
            if alpha is None or fitness[best_wolf] > fitness[alpha]:
                delta, beta, alpha = beta, alpha, best_wolf
            elif beta is None or fitness[best_wolf] > fitness[beta]:
                delta, beta = beta, best_wolf
            elif delta is None or fitness[best_wolf] > fitness[delta]:
                delta = best_wolf

    return alpha  # Best CH
def hybrid_routing(source, destination, cluster_heads, routing_table):
    """Route packets using CHs selected by GWO."""
    if source in cluster_heads.values() and destination in cluster_heads.values():
        return [source, destination]
    else:
        source_ch = cluster_heads[np.where(list(cluster_heads.values()) == source)[0][0]]
        destination_ch = cluster_heads[np.where(list(cluster_heads.values()) == destination)[0][0]]
        return [source, source_ch, destination_ch, destination]


In [9]:
import numpy as np
import random

# Parameters
num_nodes = 100  # Number of sensor nodes
max_energy = 100  # Maximum energy capacity of each node
initial_energy = 100  # Initial energy of each node
transmission_range = 30  # Transmission range of each node
energy_consumption_tx = 0.01  # Energy consumed per bit for transmission
energy_consumption_rx = 0.01  # Energy consumed per bit for reception
data_packet_size = 1000  # Size of data packet in bits
num_clusters = 5  # Number of clusters
max_gwo_iterations = 50  # Maximum iterations for GWO
num_wolves = 10  # Number of wolves in GWO

# Node class to represent each sensor node
class Node:
    def __init__(self, id, position, energy):
        self.id = id
        self.position = position
        self.energy = energy
        self.cluster_id = -1  # Cluster ID to which the node belongs

    def distance_to(self, other_node):
        return np.linalg.norm(self.position - other_node.position)

# Initialize nodes with random positions and energies
def initialize_nodes(num_nodes, initial_energy, search_space):
    nodes = []
    for i in range(num_nodes):
        position = np.array([random.uniform(search_space['x_min'], search_space['x_max']),
                            random.uniform(search_space['y_min'], search_space['y_max'])])
        nodes.append(Node(i, position, initial_energy))
    return nodes

# K-means clustering
def kmeans_clustering(nodes, num_clusters, max_iterations=100):
    # Randomly initialize cluster centroids
    centroids = np.array([nodes[random.randint(0, num_nodes - 1)].position for _ in range(num_clusters)])

    for _ in range(max_iterations):
        # Assign nodes to the nearest centroid
        for node in nodes:
            distances = [np.linalg.norm(node.position - centroid) for centroid in centroids]
            node.cluster_id = np.argmin(distances)

        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for i in range(num_clusters):
            cluster_nodes = [node.position for node in nodes if node.cluster_id == i]
            if cluster_nodes:
                new_centroids[i] = np.mean(cluster_nodes, axis=0)

        # Check for convergence
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids

    return centroids

# Grey Wolf Optimization (GWO) for cluster head selection
def gwo_optimization(nodes, num_clusters, num_wolves, max_iterations):
    # Initialize wolves (candidate solutions)
    # Change: Ensure wolf values are within the valid range
    wolves = np.array([random.sample(range(len(nodes)), num_clusters) for _ in range(num_wolves)])

    # Fitness function: minimize total energy consumption
    def fitness(wolf):
        total_energy = 0
        for cluster_id in range(num_clusters):
            cluster_head_id = wolf[cluster_id]
            # Access the cluster head using the correct index from the 'nodes' list
            cluster_head = nodes[cluster_head_id]
            for node in nodes:
                if node.cluster_id == cluster_id:
                    distance = node.distance_to(cluster_head)
                    total_energy += energy_consumption_tx * (distance ** 2)
        return total_energy
    # GWO main loop
    alpha_wolf = None
    beta_wolf = None
    delta_wolf = None
    alpha_score = float('inf')
    beta_score = float('inf')
    delta_score = float('inf')

    for iteration in range(max_iterations):
        for i in range(num_wolves):
            wolf = wolves[i]
            score = fitness(wolf)

            # Update alpha, beta, and delta wolves
            if score < alpha_score:
                alpha_score = score
                alpha_wolf = wolf
            elif score < beta_score:
                beta_score = score
                beta_wolf = wolf
            elif score < delta_score:
                delta_score = score
                delta_wolf = wolf
  # Update wolves' positions
        a = 2 - 2 * (iteration / max_iterations)  # Linearly decreases from 2 to 0
        for i in range(num_wolves):
            for j in range(num_clusters):
                A1 = 2 * a * random.random() - a
                C1 = 2 * random.random()
                D_alpha = abs(C1 * alpha_wolf[j] - wolves[i][j])
                X1 = alpha_wolf[j] - A1 * D_alpha

                A2 = 2 * a * random.random() - a
                C2 = 2 * random.random()
                D_beta = abs(C2 * beta_wolf[j] - wolves[i][j])
                X2 = beta_wolf[j] - A2 * D_beta

                A3 = 2 * a * random.random() - a
                C3 = 2 * random.random()
                D_delta = abs(C3 * delta_wolf[j] - wolves[i][j])
                X3 = delta_wolf[j] - A3 * D_delta
                # Corrected indentation:
                wolves[i][j] = int(np.clip((X1 + X2 + X3) / 3, 0, len(nodes) - 1))

    return alpha_wolf
# Main function
def main():
    # Define search space (2D area for WSN)
    search_space = {
        'x_min': 0,
        'x_max': 100,
        'y_min': 0,
        'y_max': 100
    }

    # Initialize nodes
    nodes = initialize_nodes(num_nodes, initial_energy, search_space)

    # Perform K-means clustering
    centroids = kmeans_clustering(nodes, num_clusters)
    print("K-means clustering completed.")

    # Perform GWO for cluster head selection
    best_wolf = gwo_optimization(nodes, num_clusters, num_wolves, max_gwo_iterations)
    print("GWO optimization completed.")

    # Assign cluster heads
    for cluster_id in range(num_clusters):
        cluster_head_id = best_wolf[cluster_id]
        print(f"Cluster {cluster_id + 1}: Cluster Head = Node {cluster_head_id}")

# Run the program
if __name__ == "__main__":
    main()

K-means clustering completed.
GWO optimization completed.
Cluster 1: Cluster Head = Node 6
Cluster 2: Cluster Head = Node 0
Cluster 3: Cluster Head = Node 0
Cluster 4: Cluster Head = Node 0
Cluster 5: Cluster Head = Node 0
